In [11]:
BASE_DIR ='../'*3
CHCK_DIR = BASE_DIR + 'code/UTIL/chck_device/chck_device.ipynb'

In [12]:
%run {CHCK_DIR}

Using device: cpu
Using PyTorch version: 1.9.0


In [13]:
import torch.nn as nn

In [14]:
class Model(object):
    def __init__(self, name, dataset):
        self.name = name
        self.dataset = dataset
        self.is_training = False
        if not hasattr(self, 'rand_std'): self.rand_std = 0.030
    
    def __str__(self):
        return '{}/{}'.format(self.name, self.dataset)

    def exec_all(self, epoch_count=10, batch_size=10, learning_rate=0.001,
                 report=0, show_cnt=3):
        self.train(epoch_count, batch_size, learning_rate, report)
        self.test()
        if show_cnt > 0: self.visualize(show_cnt)

In [15]:
class MlpTorch(Model):
    def __init__(self, name, dataset, hconfigs):
        super(MlpModel, self).__init__(name, dataset)
        self.make_layers(hconfigs)

In [60]:
#우선 SLP 하나로 테스트 해보고 그 뒤 추후 늘린다.
def mlp_torch_make_layer(self, hconfigs):
    self.hconfig = hconfigs
    self.layers = []
    
    prev_shape = self.dataset.input_shape
    
    for hconfig in hconfigs:
        #hconfig에는 output값이 들어있음.
        _, prev_shape=self.alloc_make_layer(prev_shape, hconfig)
        
    output_cnt = int(np.prod(self.dataset.output_shape))
    self.alloc_make_layer(prev_shape, output_cnt, atf=False)

def alloc_make_layer(self, input_shape, hconfig, atf=True):
    
    input_cnt = input_shape
    output_cnt = np.prod(hconfig)
    
    self.layer.append(nn.Linear(in_features=input_cnt,out_features=output_cnt))
    if atf:
        self.layer.append(nn.ReLU())
        return input_cnt, output_cnt
MlpTorch.make_layer = mlp_torch_make_layer
MlpTorch.alloc_make_layer = alloc_make_layer

In [39]:
class Torch_model(MlpTorch, nn.Module):
    def __init__(self):
        
        super(Torch_model, self).__init__()
        self.layer = nn.Sequential(*self.layers)
        
    def __forward__(self, x):
        out = self.layer(x)
        return out

In [62]:
def mlp_torch_train(self, epoch_count=10, batch_size=10, \
                    learning_rate=0.001, report=0):
    
    self.learning_rate = learning_rate
    
    time1 = time2 = int(time.time())
    
    if report != 0:
        print('Model {} train started'.format(self.name))
        
    # 이부분에서 dataset.Dataloader에서 train_data를 가져오면 된다. 33434
    # -> or enumerate돌아가는 부분에서 dataloader함수를 호출하여 배치 단위별로 불러온다.
    
    #  어짜피 인스턴스로 가져오는거니깐 for문 밖에서 정의 해주고 불러온다.
    # 이부분에서 그럼 self에 저장된 변수를 그냥 가져와서 전역변수로 사용한다.
    
    train_loader=self.dataset.dataloader
    self.model = Torch_model().to(DEVICE)
    for epoch in range(epoch_count + 1):
        costs = []
        accs = []
        for batch_idx, samples in enumerate(train_loader):

            X_train, y_train = samples
            
            X_train=X_train.to(DEVICE)
            y_train=y_train.to(DEVICE)
            
            optimizer.zero_grad()

            # predict
            output = model.forward(X_train)

            loss = loss_func(output, y_train)
            
            # 정확도함수 만들어야함.
            acc = get_estimate(X_train,y_train,output)
            costs.append(loss)
            
            # cost로 H(x) 계산

            loss.backward()
            optimizer.step()
            loss_arr.append(loss.cpu().detach().numpy())

        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
                epoch, nb_epochs, batch_idx+1, len(dataloader),
                loss.item()
                ))
        if report > 0 and (epoch+1) % report == 0:
            
            # get_ validate_data => 설정
            vaX, vaY = self.dataset.get_validate_data(100)
            acc = self.eval_accuracy(vaX, vaY)
            time3 = int(time.time())
            tm1, tm2 = time3-time2, time3-time1
            self.dataset.train_prt_result(epoch+1, costs, accs, acc, tm1, tm2)
            time2 = time3
            
Torch_model.train = mlp_torch_train

In [ ]:
def mlp_model_test(self):
    with torch.np_grad():
        test_loader=self.dataset.dataloader
        time1 = int(time.time())
        
        # 이부분이 이상함. 
        
        for sample in test_loader:
            X, y = sample
            X_test=X.to(DEVICE)
            y_test = y.to(DEVICE)
            
            output = self.model.forward(X_test)
            acc = self.eval_accuracy(teX, teY)
        time2 = int(time.time())
        self.dataset.test_prt_result(self.name, acc, time2-time1)

            
    teX, teY = self.dataset.get_test_data()
#     time1 = int(time.time())
    acc = self.eval_accuracy(teX, teY)
    time2 = int(time.time())
    self.dataset.test_prt_result(self.name, acc, time2-time1)

MlpModel.test = mlp_model_test

In [ ]:
# def mlp_eval_accuracy(self, x, y, output=None):
def mlp_torch_eval_accuracy(self, x, y, output):
#     if output is None:
#         output, _ = self.forward_neuralnet(x)
    accuracy = self.dataset.eval_accuracy(x, y, output)
    return accuracy

MlpModel.eval_accuracy = mlp_torch_eval_accuracy

In [ ]:
def mlp_torch_get_estimate(self, x):
    output, _ = self.forward_neuralnet(x)
    estimate = self.dataset.get_estimate(output)
    return estimate

MlpModel.get_estimate = mlp_torch_get_estimate